<a href="https://colab.research.google.com/github/xechehot/sirius-2021-warehouse/blob/main/Maksim/Correct_create_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

In [2]:
!git clone https://github.com/xechehot/sirius-2021-warehouse

Cloning into 'sirius-2021-warehouse'...
remote: Enumerating objects: 423, done.
remote: Counting objects: 100% (423/423), done.
remote: Compressing objects: 100% (317/317), done.
remote: Total 423 (delta 263), reused 225 (delta 95), pack-reused 0
Receiving objects: 100% (423/423), 415.85 KiB | 4.20 MiB/s, done.
Resolving deltas: 100% (263/263), done.


In [3]:
! cd sirius-2021-warehouse; git pull

Already up to date.


In [4]:
from json import load as JLoad

with open('sirius-2021-warehouse/data_sample_output/output_64_1_stepan.json', 'r') as f:
  data = JLoad(f) 

In [5]:
with open('sirius-2021-warehouse/data_sample/sample_64_1.json', 'r') as f:
  sample_data = JLoad(f) 

In [13]:
# bucket_item из результатов работы алгоритма выглядит так:
bucket_item_example = {
    "block_x": 1,
    "block_y": 2,
    "cnt": 1,
    "floor": 1,
    "id": 68542596,
    "row": 19,
    "section": 6
}

In [11]:
# Эта функция конвертирует bucket_item в item для order-a
def get_out_coordinates(bucket_item):
    # ТУТ НУЖНО ПРАВИЛЬНО ПОСЧИТАТЬ КООРДИНТЫ x и y из bucket_item['p']
    return {
      "position": {
        "x": 0,
        "y": 0
      },
      "id": bucket_item['id'],
      "amount": bucket_item['cnt']
    }

In [14]:
get_out_coordinates(bucket_item_example)

{'amount': 1, 'id': 68542596, 'position': {'x': 0, 'y': 0}}

In [19]:
def convert_bucket_to_items(bucket_items):
    out_items = []
    for item in bucket_items:
        out_items.append(get_out_coordinates(item))
    return out_items

## Создаем order для визуализации

In [6]:
floors = sample_data['warehouse']['meta']['floors']
block_x = sample_data['warehouse']['meta']['block_x']
block_y = sample_data['warehouse']['meta']['block_y']
rows = sample_data['warehouse']['meta']['rows'] 
sections = sample_data['warehouse']['meta']['sections']

In [7]:
def conv(place): 
    x = (sections + 2) * (place['block_x'] - 1)
    x += place['section']
    y = ((3 * rows) // 2 + 1) * (place['block_y'] - 1)
    y += (place['row'] // 2) * 3
    return [x, y]

In [8]:
def normolize_path(path, start):
    path.append([0, path[-1][1]])
    path.append(start)
    res = [path[0]]
    for i in range(1, len(path)):
        if [path[i - 1][0], path[i][1]] != res[-1]:
            res.append([path[i - 1][0], path[i][1]])
        if path[i] != res[-1]:
            res.append(path[i])
        #print(res)
    return res

In [21]:
paths = []
map_y = ((3 * rows) // 2 + 1)
map_x = (sections + 2)
sx = block_x * map_x - 1
sy = block_y * map_y // 2
for batch in data['batches']:
    for bucket in batch['buckets']:
        paths.append(bucket['items'])
ans = []
for bucket in paths:
    n_bucket = [[sx, sy]]
    for place in bucket:
        n_bucket.append(conv(place))
    ans.append(n_bucket)

output = {
    "agentsCount": 2,
    "orders": [
               
    ]
}
print(len(ans))
for bucket, bucket_items in zip(ans, paths):
    print(bucket)
    print(bucket_items)
    res = normolize_path(bucket, [sx, sy])
    bucket = {
        # Сюда нужно добавить item из bucket_items
        "items": convert_bucket_to_items(bucket_items),
        "path": [
             
        ]
    }
    for x, y in res:
        place = {
            "x": x,
            "y": y
        }
        bucket['path'].append(place)
    output['orders'].append(bucket)
#print(json.dumps(output, indent=2))
with open("visual_sample_order.json", "w") as f:
    json.dump(output, f, indent=2)         

10
[[9, 7], [7, 3], [2, 3], [1, 3], [3, 3], [1, 3]]
[{'block_x': 1, 'block_y': 1, 'cnt': 4, 'floor': 1, 'id': 82408055, 'row': 3, 'section': 7}, {'block_x': 1, 'block_y': 1, 'cnt': 3, 'floor': 1, 'id': 80476859, 'row': 3, 'section': 2}, {'block_x': 1, 'block_y': 1, 'cnt': 1, 'floor': 1, 'id': 27038923, 'row': 3, 'section': 1}, {'block_x': 1, 'block_y': 1, 'cnt': 1, 'floor': 1, 'id': 88906902, 'row': 2, 'section': 3}, {'block_x': 1, 'block_y': 1, 'cnt': 2, 'floor': 1, 'id': 22895628, 'row': 2, 'section': 1}]
[[9, 7], [7, 3], [3, 3], [3, 3], [1, 3]]
[{'block_x': 1, 'block_y': 1, 'cnt': 4, 'floor': 1, 'id': 82408055, 'row': 3, 'section': 7}, {'block_x': 1, 'block_y': 1, 'cnt': 1, 'floor': 1, 'id': 88906902, 'row': 2, 'section': 3}, {'block_x': 1, 'block_y': 1, 'cnt': 1, 'floor': 1, 'id': 56193990, 'row': 3, 'section': 3}, {'block_x': 1, 'block_y': 1, 'cnt': 2, 'floor': 1, 'id': 22895628, 'row': 2, 'section': 1}]
[[9, 7], [7, 0]]
[{'block_x': 1, 'block_y': 1, 'cnt': 6, 'floor': 1, 'id': 88